In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16267274
paper_name = 'dunn_jensen_2006' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/Supp_Table3.xlsx', sheet_name='Sheet1', skiprows=11)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 3792 x 6


In [10]:
original_data.head()

,Systemic name,Gene,(-EtBr/+EtBr),plating to EtBr,EtBr treatment,combination with mrpl16D)
0,NaN,NaN,NaN,NaN,NaN,NaN
1,YJR120W,*ATP2,4.20,X,X,X
2,YHR025W,THR1,3.95,X,X,X
3,YPR197C,NaN,3.88,X,NaN,NaN
4,YIL098C,FMC1,3.39,X,X,X


In [11]:
original_data['orf'] = original_data['Systemic name'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            Systemic name Gene  (-EtBr/+EtBr) plating to EtBr  EtBr treatment  \
index_input                                                                     
0                     NaN  NaN            NaN             NaN             NaN   

             combination with mrpl16D)  orf  
index_input                                  
0                                  NaN  NAN  


In [15]:
original_data = original_data.loc[t,:]

In [16]:
original_data['data'] = 1 / pd.to_numeric(original_data['(-EtBr/+EtBr)'], errors='coerce')

In [17]:
original_data.set_index('orf', inplace=True)

In [18]:
original_data = original_data[['data']].copy()

In [19]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data.shape

(3752, 1)

# Prepare the final dataset

In [21]:
data = original_data.copy()

In [22]:
dataset_ids = [54]
datasets = datasets.reindex(index=dataset_ids)

In [23]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [24]:
data.head()

dataset_id,54
data_type,value
orf,
YAL012W,-0.925926
YAL024C,1245.330012
YAL054C,5.235602
YAL056C-A,-0.406504
YAL064C-A,-4.524887


## Subset to the genes currently in SGD

In [25]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 11


In [26]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,54
,data_type,value
gene_id,orf,
10,YAL012W,-0.925926
22,YAL024C,1245.330012
50,YAL054C,5.235602
1866,YAL056C-A,-0.406504
1867,YAL064C-A,-4.524887


# Normalize

In [27]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [28]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [29]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id                  54           
data_type                value     valuez
gene_id orf                              
10      YAL012W      -0.925926  -0.128084
22      YAL024C    1245.330012  11.822208
50      YAL054C       5.235602  -0.069001
1866    YAL056C-A    -0.406504  -0.123103
1867    YAL064C-A    -4.524887  -0.162594

# Print out

In [30]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [31]:
from IO.save_data_to_db3 import *

In [32]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 16267274...
Inserting the new data...


100%|██████████| 1/1 [00:05<00:00,  5.55s/it]

Updating the data_modified_on field...
